In [1]:
# DNNベース手法　教師なし学習

In [2]:
## ライブラリやモジュールの読み込み
from glob import glob
from input_output import input, output
from dnn_tools import preprocess_dnn, dataloaders_unsupervised
from dnn_tools import train_unsupervised, pred_unsupervised, LSTM

In [3]:
# データの読み込み
# データファイルの検索
folder_path = "./data"
noise_lebel = 0.5
train_normaly_file_path_list  = glob(f"{folder_path}/train/normaly_{noise_lebel}/*.csv")
test_amplitude_file_path      = glob(f"{folder_path}/test/anormaly_{noise_lebel}/local_amplitude_00001.csv")
test_average_file_path        = glob(f"{folder_path}/test/anormaly_{noise_lebel}/local_average_00001.csv")
test_time_amplitude_file_path = glob(f"{folder_path}/test/anormaly_{noise_lebel}/time_series_amplitude_00001.csv")
test_time_average_file_path   = glob(f"{folder_path}/test/anormaly_{noise_lebel}/time_series_average_00001.csv")
test_time_frequency_file_path = glob(f"{folder_path}/test/anormaly_{noise_lebel}/time_series_frequency_00001.csv")

# データファイルの読み込み
train_normaly_list       = input(train_normaly_file_path_list)
test_amplitude_list      = input(test_amplitude_file_path)
test_average_list        = input(test_average_file_path)
test_time_amplitude_list = input(test_time_amplitude_file_path)
test_time_average_list   = input(test_time_average_file_path)
test_time_frequency_list = input(test_time_frequency_file_path)

In [4]:
# 前処理
train_normaly_data_df_list       = preprocess_dnn(train_normaly_list)
test_amplitude_data_df_list      = preprocess_dnn(test_amplitude_list)
test_average_data_df_list        = preprocess_dnn(test_average_list)
test_time_amplitude_data_df_list = preprocess_dnn(test_time_amplitude_list)
test_time_average_data_df_list   = preprocess_dnn(test_time_average_list)
test_time_frequency_df_list      = preprocess_dnn(test_time_frequency_list)

# データローダー化
window_size = 10
train_normaly_data_loader, val_normaly_label_loader        = dataloaders_unsupervised(train_normaly_data_df_list, window_size)
test_amplitude_data_loader, test_amplitude_label           = dataloaders_unsupervised(test_amplitude_data_df_list , window_size, train=False)
test_average_data_loader, test_average_label               = dataloaders_unsupervised(test_average_data_df_list , window_size, train=False)
test_time_amplitude_data_loader, test_time_amplitude_label = dataloaders_unsupervised(test_time_amplitude_data_df_list , window_size, train=False)
test_time_average_data_loader, test_time_average_label     = dataloaders_unsupervised(test_time_average_data_df_list , window_size, train=False)
test_time_frequency_data_loader, test_time_frequency_label = dataloaders_unsupervised(test_time_frequency_df_list  , window_size, train=False)

test_data_loader_label_dict = {
    "amplitude"     :[test_amplitude_data_loader, test_amplitude_label],
    "average"       :[test_average_data_loader, test_average_label],
    "time_amplitude":[test_time_amplitude_data_loader, test_time_amplitude_label],
    "time_average"  :[test_time_average_data_loader, test_time_average_label],
    "time_frequency":[test_time_frequency_data_loader, test_time_frequency_label]
}

In [5]:
# モデル学習
# モデル選定
input_size = len(train_normaly_data_df_list[0].columns) -1
model = LSTM(input_size)
# 学習
train_model = train_unsupervised(train_normaly_data_loader, val_normaly_label_loader, model)

Epoch [1/20], Train Loss: 0.0450, Val Loss: 0.0006
Epoch [2/20], Train Loss: 0.0005, Val Loss: 0.0003
Epoch [3/20], Train Loss: 0.0003, Val Loss: 0.0004
Epoch [4/20], Train Loss: 0.0003, Val Loss: 0.0002
Epoch [5/20], Train Loss: 0.0003, Val Loss: 0.0003
Epoch [6/20], Train Loss: 0.0005, Val Loss: 0.0003
Epoch [7/20], Train Loss: 0.0003, Val Loss: 0.0003
Epoch [8/20], Train Loss: 0.0005, Val Loss: 0.0014
Epoch [9/20], Train Loss: 0.0008, Val Loss: 0.0007
Epoch [10/20], Train Loss: 0.0006, Val Loss: 0.0008
Epoch [11/20], Train Loss: 0.0009, Val Loss: 0.0009
Epoch [12/20], Train Loss: 0.0006, Val Loss: 0.0002
Epoch [13/20], Train Loss: 0.0004, Val Loss: 0.0010
Epoch [14/20], Train Loss: 0.0008, Val Loss: 0.0008
Epoch [15/20], Train Loss: 0.0006, Val Loss: 0.0006
Epoch [16/20], Train Loss: 0.0008, Val Loss: 0.0020
Epoch [17/20], Train Loss: 0.0006, Val Loss: 0.0003
Epoch [18/20], Train Loss: 0.0004, Val Loss: 0.0012
Epoch [19/20], Train Loss: 0.0008, Val Loss: 0.0001
Epoch [20/20], Train 

In [6]:
# 異常検知
for ano_name in test_data_loader_label_dict:
    test_data_loader, test_label = test_data_loader_label_dict[ano_name]
    prediction = pred_unsupervised(train_model, test_data_loader)
    output(prediction, test_label, output_file_path=f"./result/lstm_unsupervised_{ano_name}_{noise_lebel}.csv")